In [136]:
#imports
import pandas as pd
import numpy as np
import csv
import random

In [137]:
#constants

stats = [
    'W',
    'L',
    'n_played',
    'H_W',
    'H_L',
    'A_W',
    'A_L',
    'N_W',
    'N_L',
    'avg_ppg',
    'avg_ppga'
]

pt_stats = [
    'avg_fgm',
    'avg_fga',
    'avg_fgm3',
    'avg_fga3',
    'avg_ftm',
    'avg_fta',
    'avg_or',
    'avg_dr',
    'avg_ast',
    'avg_to',
    'avg_stl',
    'avg_blk',
    'avg_pf'
]

game_types = [
    'W',
    'L',
    'H_W',
    'H_L',
    'A_W',
    'A_L',
    'N_W',
    'N_L'
]

stats.extend(pt_stats)

lastN = 8
recent_stats = [ s+'_last_'+str(lastN) for s in stats ]
stats.extend(recent_stats)

opp = {
    'W': 'L',
    'L': 'W'
}

seasons = [ yr for yr in range(2003,2018) ]
rnd = '2'

print('Using features:')
print(stats + recent_stats)

Using features:
['W', 'L', 'n_played', 'H_W', 'H_L', 'A_W', 'A_L', 'N_W', 'N_L', 'avg_ppg', 'avg_ppga', 'avg_fgm', 'avg_fga', 'avg_fgm3', 'avg_fga3', 'avg_ftm', 'avg_fta', 'avg_or', 'avg_dr', 'avg_ast', 'avg_to', 'avg_stl', 'avg_blk', 'avg_pf', 'W_last_8', 'L_last_8', 'n_played_last_8', 'H_W_last_8', 'H_L_last_8', 'A_W_last_8', 'A_L_last_8', 'N_W_last_8', 'N_L_last_8', 'avg_ppg_last_8', 'avg_ppga_last_8', 'avg_fgm_last_8', 'avg_fga_last_8', 'avg_fgm3_last_8', 'avg_fga3_last_8', 'avg_ftm_last_8', 'avg_fta_last_8', 'avg_or_last_8', 'avg_dr_last_8', 'avg_ast_last_8', 'avg_to_last_8', 'avg_stl_last_8', 'avg_blk_last_8', 'avg_pf_last_8', 'W_last_8', 'L_last_8', 'n_played_last_8', 'H_W_last_8', 'H_L_last_8', 'A_W_last_8', 'A_L_last_8', 'N_W_last_8', 'N_L_last_8', 'avg_ppg_last_8', 'avg_ppga_last_8', 'avg_fgm_last_8', 'avg_fga_last_8', 'avg_fgm3_last_8', 'avg_fga3_last_8', 'avg_ftm_last_8', 'avg_fta_last_8', 'avg_or_last_8', 'avg_dr_last_8', 'avg_ast_last_8', 'avg_to_last_8', 'avg_stl_last_8'

In [138]:
#read original data
data = pd.read_csv('data/RegularSeasonDetailedResults.csv')
team_data = pd.read_csv('data/Teams.csv')
seeds = pd.read_csv('data/TourneySeeds.csv')

In [139]:
for season in seasons:
    data = pd.read_csv('data/RegularSeasonDetailedResults.csv')
    unique_teams = pd.concat([data["Wteam"],data["Lteam"]]).unique()
    table_data = {}
    for team in unique_teams:
        table_data[team] = np.zeros(unique_teams.max()+1)
    table = pd.DataFrame(data=table_data,columns=unique_teams)
    table.to_csv('data/processed/tables/'+str(season)+'.csv',index=False)

In [140]:
tables = {}
for season in seasons:
    tables[str(season)] = pd.read_csv('data/processed/tables/'+str(season)+'.csv')

In [141]:
with open('data/processed/stats.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['season','team','seed'] + stats)

#generate season stats for each team
for season in seasons:
    team_stats = {}
    recent_team_stats = {}
    year_data = data.loc[data['Season'] == season]
    year_seeds = seeds.loc[seeds['Season'] == season]
    for index,row in year_data.iterrows():
        tables[str(season)].iloc[row['Wteam']][str(row['Lteam'])] += 1
        tables[str(season)].iloc[row['Lteam']][str(row['Wteam'])] -= 1
        
        for res in ['W','L']:
            team = row[res+'team']
            
            if team not in team_stats.keys():
                team_stats[team] = {}
                team_stats[team]['last_n_rows'] = {}
                for s in stats:    
                    team_stats[team][s] = 0
                    if s not in recent_stats:
                        team_stats[team]['last_n_rows'][s] = [] 
            
            n_played = team_stats[team]['n_played']
            for ps in pt_stats:
                team_stats[team][ps] *= n_played
                team_stats[team][ps] += row[res+ps.split('_')[1]]
                team_stats[team][ps] /= n_played+1.0
                team_stats[team]['last_n_rows'][ps].append(row[res+ps.split('_')[1]])

            
            team_stats[team]['avg_ppg'] *= n_played
            team_stats[team]['avg_ppg'] += row[res+'score']
            team_stats[team]['avg_ppg'] /= n_played+1.0
            team_stats[team]['last_n_rows']['avg_ppg'].append(row[res+'score'])

            
            team_stats[team]['avg_ppga'] *= n_played
            team_stats[team]['avg_ppga'] += row[opp[res]+'score']
            team_stats[team]['avg_ppga'] /= n_played+1.0
            team_stats[team]['last_n_rows']['avg_ppga'].append(row[opp[res]+'score'])

            
            team_stats[team][res] += 1
            team_stats[team][row['Wloc']+'_'+res] += 1
            
            for game_type in game_types:
                if game_type == row['Wloc']+'_'+res or game_type == res:
                    team_stats[team]['last_n_rows'][game_type].append(1)
                else:
                    team_stats[team]['last_n_rows'][game_type].append(0)
            
            team_stats[team]['n_played'] += 1
            team_stats[team]['last_n_rows']['n_played'].append(1)

    for team in team_stats.keys():
        for s in stats:
            if s not in recent_stats:
                if s not in game_types:
                    team_stats[team][s+'_last_'+str(lastN)] = np.mean(team_stats[team]['last_n_rows'][s][-8:])
                else:
                    team_stats[team][s+'_last_'+str(lastN)] = sum(team_stats[team]['last_n_rows'][s][-8:])
        
    with open('data/processed/stats.csv', 'a') as csv_file:
        writer = csv.writer(csv_file)
        for team in team_stats.keys():
            row = [season,team]
            seed = year_seeds.loc[year_seeds['Team'] == team]
            if seed.shape[0] == 1:
                seed_num = int(''.join([ c for c in seed.iloc[0]['Seed'] if c.isdigit() ]))
                row.append(seed_num)
            else:
                row.append(16)
            for s in stats:
                row.append(team_stats[team][s])
            writer.writerow(row)

In [142]:
for season in seasons:
    tables[str(season)].to_csv('data/processed/tables/'+str(season)+'.csv',index=False)

In [143]:
#read tournament data
tourney_data = pd.read_csv('data/TourneyCompactResults.csv')
season_stats = pd.read_csv('data/processed/stats.csv')
team_names = pd.read_csv('data/Teams.csv')
team_rpi = pd.read_csv('data/processed/rpi.csv')
team_sos = pd.read_csv('data/processed/sos.csv')
unplayed = pd.read_csv('data/rounds/'+rnd+'.csv')

In [144]:
with open('data/processed/labeled.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['day','team_1','team_2','rpi_1','rpi_2','sos_1','sos_2','history'] + \
                    ['seed_1'] + [ s+'_1' for s in stats ] + \
                    ['seed_2'] + [ s+'_2' for s in stats ] + \
                    ['result'] \
                   )
    
    for index, row in tourney_data.iterrows():
        season = row['Season']
        if season in seasons:
            Wteam = row['Wteam']
            Wteam_name = team_names.loc[team_names['Team_Id'] == Wteam].iloc[0]['Team_Name']
            
            Lteam = row['Lteam']
            Lteam_name = team_names.loc[team_names['Team_Id'] == Lteam].iloc[0]['Team_Name']
            
            day = row['Daynum']
        
            Wrpi_data = team_rpi.loc[(team_rpi['team'].str.strip() == Wteam_name) & (team_rpi['season'] == season)]
            Wrpi = Wrpi_data.iloc[0]['rpi'] if Wrpi_data.shape[0] > 0 else team_rpi['rpi'].mean()
            Lrpi_data = team_rpi.loc[(team_rpi['team'].str.strip() == Lteam_name) & (team_rpi['season'] == season)]
            Lrpi = Lrpi_data.iloc[0]['rpi'] if Lrpi_data.shape[0] > 0 else team_rpi['rpi'].mean()
            
            Wsos_data = team_sos.loc[(team_sos['team'].str.strip() == Wteam_name) & (team_sos['season'] == season)]
            Wsos = Wsos_data.iloc[0]['sos'] if Wsos_data.shape[0] > 0 else team_sos['sos'].mean()
            Lsos_data = team_sos.loc[(team_sos['team'].str.strip() == Lteam_name) & (team_sos['season'] == season)]
            Lsos = Lsos_data.iloc[0]['sos'] if Lsos_data.shape[0] > 0 else team_sos['sos'].mean()
            
            Whistory = tables[str(season)].iloc[Wteam][str(Lteam)]
            Lhistory = tables[str(season)].iloc[Lteam][str(Wteam)]
            
            Wteam_data = season_stats.loc[season_stats['season'] == season]
            Wteam_data = Wteam_data.loc[season_stats['team'] == Wteam].iloc[0]
            Lteam_data = season_stats.loc[season_stats['season'] == season]
            Lteam_data = Lteam_data.loc[season_stats['team'] == Lteam].iloc[0]

            if random.random() > 0.5:
                writer.writerow([day,Wteam_name,Lteam_name,Wrpi,Lrpi,Wsos,Lsos,Whistory] + \
                                list(Wteam_data)[2:] + \
                                list(Lteam_data)[2:] + \
                                ['win'] \
                               )
            else:
                writer.writerow([day,Lteam_name,Wteam_name,Lrpi,Wrpi,Lsos,Wsos,Lhistory] + \
                                list(Lteam_data)[2:] + \
                                list(Wteam_data)[2:] + \
                                ['lose'] \
                               )

In [145]:
with open('data/2017/'+rnd+'.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['day','team_1','team_2','rpi_1','rpi_2','sos_1','sos_2','history'] + \
                    ['seed_1'] + [ s+'_1' for s in stats ] + \
                    ['seed_2'] + [ s+'_2' for s in stats ] + \
                    ['result'] \
                   )
    
    for index, row in unplayed.iterrows():
        season = 2017
        if season in seasons:
            Wteam_name = row['Wteam']
            Wteam = team_names.loc[team_names['Team_Name'] == Wteam_name].iloc[0]['Team_Id']
            
            Lteam_name = row['Lteam']
            Lteam = team_names.loc[team_names['Team_Name'] == Lteam_name].iloc[0]['Team_Id']
            
            day = 134+random.random()
        
            Wrpi_data = team_rpi.loc[(team_rpi['team'].str.strip() == Wteam_name) & (team_rpi['season'] == season)]
            Wrpi = Wrpi_data.iloc[0]['rpi'] if Wrpi_data.shape[0] > 0 else team_rpi['rpi'].mean()
            Lrpi_data = team_rpi.loc[(team_rpi['team'].str.strip() == Lteam_name) & (team_rpi['season'] == season)]
            Lrpi = Lrpi_data.iloc[0]['rpi'] if Lrpi_data.shape[0] > 0 else team_rpi['rpi'].mean()
            
            Wsos_data = team_sos.loc[(team_sos['team'].str.strip() == Wteam_name) & (team_sos['season'] == season)]
            Wsos = Wsos_data.iloc[0]['sos'] if Wsos_data.shape[0] > 0 else team_sos['sos'].mean()
            Lsos_data = team_sos.loc[(team_sos['team'].str.strip() == Lteam_name) & (team_sos['season'] == season)]
            Lsos = Lsos_data.iloc[0]['sos'] if Lsos_data.shape[0] > 0 else team_sos['sos'].mean()
            
            Whistory = tables[str(season)].iloc[Wteam][str(Lteam)]
            Lhistory = tables[str(season)].iloc[Lteam][str(Wteam)]
            
            Wteam_data = season_stats.loc[season_stats['season'] == season]
            Wteam_data = Wteam_data.loc[season_stats['team'] == Wteam].iloc[0]
            Lteam_data = season_stats.loc[season_stats['season'] == season]
            Lteam_data = Lteam_data.loc[season_stats['team'] == Lteam].iloc[0]

            writer.writerow([day,Wteam_name,Lteam_name,Wrpi,Lrpi,Wsos,Lsos,Whistory] + \
                            list(Wteam_data)[2:] + \
                            list(Lteam_data)[2:] + \
                            ['win'] \
                           )
            writer.writerow([day,Lteam_name,Wteam_name,Lrpi,Wrpi,Lsos,Wsos,Lhistory] + \
                            list(Lteam_data)[2:] + \
                            list(Wteam_data)[2:] + \
                            ['lose'] \
                           )

In [146]:
#normalize data
labeled = pd.read_csv('data/processed/labeled.csv')
for col in labeled.columns:
    if col not in ['team_1','team_2','seed_1','seed_2','result']:
        df = labeled[col]
        labeled[col] = (df - df.mean()) / (df.max() - df.min())
del labeled['n_played_last_'+str(lastN)+'_1']
del labeled['n_played_last_'+str(lastN)+'_2']
labeled.to_csv('data/processed/normalized.csv',index=False)

#normalize data
unplayed = pd.read_csv('data/2017/'+rnd+'.csv')
for col in unplayed.columns:
    if col not in ['team_1','team_2','seed_1','seed_2','result']:
        df = unplayed[col]
        unplayed[col] = (df - df.mean()) / (df.max() - df.min())
del unplayed['n_played_last_'+str(lastN)+'_1']
del unplayed['n_played_last_'+str(lastN)+'_2']
unplayed.to_csv('data/2017/'+rnd+'_n.csv',index=False)

In [147]:
# if False:
#     best_attributes = \
#     ['rpi_1',
#      'rpi_2',
#      'seed_1',
#      'sos_1',
#      'avg_ppga_1',
#      'sos_2',
#      'day',
#      'W_1',
#      'avg_or_1',
#      'avg_fga_1',
#      'avg_fgm_1',
#      'avg_fga3_1',
#      'seed_2',
#      'avg_blk_1',
#      'avg_stl_1',
#      'avg_pf_1',
#      'avg_ppg_1',
#      'avg_fta_1',
#      'avg_fgm3_1',
#      'avg_ftm_1']

#     shuffled = pd.read_csv('data/processed/shuffled.csv')
#     for col in shuffled.columns:
#         if not (col in best_attributes or col == 'result'):
#             del shuffled[col]

#     shuffled.to_csv('data/processed/pruned.csv',index=False)

In [148]:
# shuffled = pd.read_csv('data/processed/shuffled.csv')
# numCorrect = shuffled.loc[(shuffled['seed_1'] < shuffled['seed_2']) & (shuffled['result'] == 'win')].shape[0] + \
#                 shuffled.loc[(shuffled['seed_1'] > shuffled['seed_2']) & (shuffled['result'] == 'lose')].shape[0] + \
#                 shuffled.loc[(shuffled['seed_1'] == shuffled['seed_2'])].shape[0]/2
# print(numCorrect,'out of',shuffled.shape[0])
# print('baseline is',numCorrect/shuffled.shape[0])

In [149]:
data = pd.read_csv('data/processed/normalized.csv')
del data['team_1']
del data['team_2']
del data['seed_1']
del data['seed_2']
data.to_csv('data/processed/normalized.csv',index=False)

In [150]:
matchups = pd.read_csv('data/2017/'+rnd+'_n.csv')
predictions = pd.read_csv('data/predictions.csv')
pairs = pd.read_csv('data/rounds/'+rnd+'.csv')

pairs.rounds += 1
for index,row in predictions.iterrows():
    pred = row['predicted']
    team1 = matchups.iloc[index]['team_1']
    team2 = matchups.iloc[index]['team_2']
#     print(team1,team2,pred)
    if 'lose' in pred:
        if pairs[pairs.Wteam == team1].shape[0] > 0:
            pairs.Wwins[pairs.Wteam == team1] += 1
            pairs.Wscores[pairs.Wteam == team1] += row['prediction']
            pairs.Lscores[pairs.Wteam == team1] += 1-row['prediction']
            
        elif pairs[(pairs.Lteam == team1)].shape[0] > 0:
            pairs.Lwins[pairs.Lteam == team1] += 1
            pairs.Lscores[pairs.Lteam == team1] += row['prediction']
            pairs.Wscores[pairs.Lteam == team1] += 1-row['prediction']
            
    else:
        if pairs[pairs.Wteam == team1].shape[0] > 0:
            pairs.Lwins[pairs.Wteam == team1] += 1
            pairs.Lscores[pairs.Wteam == team1] += row['prediction']
            pairs.Wscores[pairs.Wteam == team1] += 1-row['prediction']
            
        elif pairs[pairs.Lteam == team1].shape[0] > 0:
            pairs.Wwins[pairs.Lteam == team1] += 1
            pairs.Wscores[pairs.Lteam == team1] += row['prediction']
            pairs.Lscores[pairs.Lteam == team1] += 1-row['prediction']


            
pairs.to_csv('data/rounds/'+rnd+'.csv',index=False)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th